In [63]:
import os
import pandas as pd

In [22]:
katy_data_path_eras = "../../Data/cholangio/Erasmus/tumors/"
katy_data_path_msk = "../../Data/cholangio/MSK/tumor/"

amber_data_path = "../../Data/amber_data"

In [41]:
katy_files_eras = os.listdir(katy_data_path_eras)
katy_files_msk = os.listdir(katy_data_path_msk)

katy_files = katy_files_eras + katy_files_msk

katy_files_mhd = [x for x in katy_files if "mhd" in x]
katy_files_raw = [x for x in katy_files if "raw" in x]

print("Katy mhd: ", len(katy_files_mhd))
print("Katy raw: ", len(katy_files_raw))

Katy mhd:  246
Katy raw:  246


In [42]:
amber_files = os.listdir(amber_data_path)

amber_files_mhd = [x for x in amber_files if "mhd" in x]
amber_files_raw = [x for x in amber_files if "raw" in x]

print("Amber mhd: ", len(amber_files_mhd))
print("Amber raw: ", len(amber_files_raw))

Amber mhd:  279
Amber raw:  279


In [58]:
set_amb_files_mhd = set(amber_files_mhd)
set_kt_files_mhd = set(katy_files_mhd)
set_moved_files_mhd = set(cholAmb_files_mhd)

kt_missing_files = (set_amb_files_mhd ^ set_kt_files_mhd)&set_amb_files_mhd

print("Katy missing count: ", len(kt_missing_files))

Katy missing count:  67


In [55]:
chol_moved = os.listdir("../../Data/cholangio/Amber")

chol_moved_files_mhd = [x for x in chol_moved if "mhd" in x]
chol_moved_files_raw = [x for x in chol_moved if "raw" in x]
print("mhd: ", len(chol_moved_files_mhd))
print("raw: ", len(chol_moved_files_raw))

mhd:  67
raw:  67


In [99]:
all_patient_files = katy_files + chol_moved

all_patients = [x for x in all_patient_files if "mhd" in x]
print(len(all_patients))

ver1_patient_ids = [x[:-10] for x in all_patients if "_Tumor.mhd" in x]

ver2_patient_ids = [x[:-4] for x in all_patients if "_Tumor.mhd" not in x]

all_patient_ids = ver1_patient_ids + ver2_patient_ids

all_patient_ids_tbl = pd.DataFrame({'ScoutID':all_patient_ids})
all_patient_ids_tbl.to_excel("../../Data/all_patients.xlsx")

313


In [68]:
# find out which no_genetics patients have RFS labels
no_genetics_tbl = pd.read_excel("../../Data/nogeneticdata_patients.xlsx")

rfs_labelled_tbl = pd.read_excel("../../Data/RFS_Scout.xlsx")

In [70]:
nogene_wrfs = pd.merge(no_genetics_tbl, rfs_labelled_tbl, how='inner', on=['ScoutID'])
nogene_wrfs.to_excel("../../Data/no_genes_with_rfs.xlsx")

In [73]:
rfs_and_genes_tbl = pd.read_excel("../../Data/labelled_Genomic_Data.xlsx")
should_be_empty = pd.merge(nogene_wrfs, rfs_and_genes_tbl, on=['ScoutID'])

In [110]:
MSK_genomic_data = pd.read_excel("../../Data/MSK_Genomic_Data.xlsx")
MSK_genomic_data['ScoutID'] = MSK_genomic_data['ScoutID'].str.strip()
pats_w_genes = pd.merge(MSK_genomic_data, all_patient_ids_tbl, on=['ScoutID'])
print("Patients w/ images and genes: ", len(pats_w_genes.ScoutID))

pats_w_genes_and_labels = pd.merge(pats_w_genes, rfs_labelled_tbl, on=['ScoutID'])
print("Patients w/ images, genes, and RFS labels: ", len(pats_w_genes_and_labels.ScoutID))

Patients w/ images and genes:  136
Patients w/ images, genes, and RFS labels:  111


In [119]:
im_and_gene = set(MSK_genomic_data['ScoutID'].tolist())
im_gene_and_label = set(pats_w_genes_and_labels['ScoutID'].tolist())

(im_and_gene ^ im_gene_and_label) & im_and_gene

{'011_ICCradio',
 '012_ICCradio',
 '013_ICCradio',
 '014_ICCradio',
 '015_ICCradio',
 '016_ICCradio',
 '017_ICCradio',
 '020_ICCradio',
 '021_ICCradio',
 '026_ICCradio',
 '027_ICCradio',
 '028_ICCradio',
 '030_ICCradio',
 '032_ICCradio',
 '033_ICCradio',
 '034_ICCradio',
 '035_ICCradio',
 '036_ICCradio',
 '037_ICCradio',
 '038_ICCradio',
 '040_ICCradio',
 '041_ICCradio',
 '043_ICCradio',
 '045_ICCradio',
 '047_ICCradio'}

In [129]:
# Finding patients with no genetics and no rfs
# all_patient_ids - list
# MSK_genomic_data - pd dataframe
# rfs_labelled_tbl - pd dataframe

pats = set(all_patient_ids)
genes = set(MSK_genomic_data['ScoutID'].tolist())
labels = set(rfs_labelled_tbl['ScoutID'].tolist())

# no genes
pats_no_genes = (pats ^ genes) & pats

# no genes and labels
pats_no_genes_no_labels = (pats_no_genes ^ labels) & pats_no_genes

len(pats_no_genes_no_labels)

pats_nogenes_nolabels_tbl = pd.DataFrame({'ScoutID':list(pats_no_genes_no_labels)})
pats_nogenes_nolabels_tbl.sort_values('ScoutID', inplace=True)
pats_nogenes_nolabels_tbl.to_excel("../../Data/images_only_data.xlsx")

In [124]:
313-210

103

In [134]:
should_be_empty = pd.merge(pats_nogenes_nolabels_tbl,nogene_wrfs, on=['ScoutID'])
should_be_empty

,ScoutID,RFS_Code,RFS


In [138]:
211-137

74

In [139]:
112-1

111

In [140]:
111+74

185